In [1]:
import geopandas as gpd
import pandas as pd

## Open State data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_gdf = gpd.read_file(file_path)
states_gdf["NAME"] = states_gdf["NAME"].str.title()
states_gdf = states_gdf[["geometry", "NAME"]]

## Open Energy Data

In [3]:
state_total_energy_record_names = [
    f"{state_name} : all fuels (utility-scale)"
    for state_name in list(states_gdf["NAME"])
]

In [4]:
state_natural_gas_energy_record_names = [
    f"{state_name.title()} : natural gas" for state_name in list(states_gdf["NAME"])
]

In [5]:
energy_df = pd.read_csv("data/Net_generation_for_all_sectors.csv")

In [6]:
def format_natural_gas_col(row) -> int:
    natural_gas = row["2023"]
    if natural_gas.isnumeric():
        return int(natural_gas)
    else:
        print(row["description"], row["2023"])
        return 0

In [7]:
natural_gas_df = energy_df[
    energy_df["description"].isin(state_natural_gas_energy_record_names)
]
natural_gas_df["NAME"] = natural_gas_df["description"].str.split(":").str[0]
total_df["NATURAL_GAS"] = natural_gas_df.apply(
    lambda row: format_natural_gas_col(row), axis=1
)
natural_gas_df = natural_gas_df[["NAME", "NATURAL_GAS"]].reset_index(drop=True)

Hawaii : natural gas --


/tmp/ipykernel_11856/3860401195.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  natural_gas_df['NAME'] = natural_gas_df['description'].str.split(':').str[0]
/tmp/ipykernel_11856/3860401195.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  natural_gas_df['NATURAL_GAS'] = natural_gas_df.apply(lambda row: format_natural_gas_col(row), axis=1)


In [8]:
total_df = energy_df[energy_df["description"].isin(state_total_energy_record_names)]
total_df["NAME"] = total_df["description"].str.split(":").str[0]
total_df["TOTAL"] = total_df["2023"].astype(int)
total_df = total_df[["NAME", "TOTAL"]].reset_index(drop=True)

/tmp/ipykernel_11856/1688030734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df['NAME'] = total_df['description'].str.split(':').str[0]
/tmp/ipykernel_11856/1688030734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df['TOTAL'] = total_df['2023'].astype(int)


In [9]:
formatted_energy_df = natural_gas_df.merge(total_df, how="inner", on="NAME")

In [10]:
formatted_energy_df["percent"] = (
    formatted_energy_df["NATURAL_GAS"] / formatted_energy_df["TOTAL"]
).round(decimals=3) * 100

## Merge Data

In [11]:
formatted_energy_df["NAME"] = formatted_energy_df["NAME"].str.strip()
states_gdf["NAME"] = states_gdf["NAME"].str.strip()

In [15]:
energy_per_state_gdf = states_gdf.merge(formatted_energy_df, on="NAME", how="inner")

In [17]:
energy_per_state_gdf = energy_per_state_gdf.to_crs(9311)
energy_per_state_gdf.to_file("data/energy.gpkg")